## Schritt 1: Word2vec-Index herunterladen

https://devmount.github.io/GermanWordEmbeddings/

In [2]:
from tqdm import tqdm_notebook as tqdm 
import requests
import math
import os
import zipfile

if not os.path.exists("german.model"):

    url = "https://downloads.codingcoursestv.eu/037%20-%20neuronale%20netze/german.model.zip"
    # Streaming, so we can iterate over the response.
    r = requests.get(url, stream=True)

    # Total size in bytes.
    total_size = int(r.headers.get('content-length', 0));
    
    print("Downloading " + str(total_size / 1024 / 1024) + " MB.")
    
    block_size = 1024
    with open('german.model.zip', 'wb') as f:
        for data in tqdm(r.iter_content(block_size), total=math.ceil(total_size//block_size), unit='KB', unit_divisor=1024, unit_scale=True):
            f.write(data)
            
    # Extracting .zipFile
    with zipfile.ZipFile("german.model.zip", "r") as zipf:
        zipf.extract("german.model")
        
    # Remove .zip-File (we don't need it anymore)
    os.remove("german.model.zip")
    
    print("Done!")
else:
    print("Datei existiert bereits")

Datei existiert bereits


In [3]:
# word2vec (gensim): https://radimrehurek.com/gensim/models/word2vec.html

In [4]:
from gensim.models import KeyedVectors

wv = KeyedVectors.load_word2vec_format("german.model", binary=True)

Funktion des Word2Vec

In [5]:
# Obama - USA + Russland = Putin

wv.most_similar(positive=["Obama", "Russland"], negative=["USA"])

[('Putin', 0.7802654504776001),
 ('US-Praesident_Obama', 0.732889711856842),
 ('Barack_Obama', 0.7064037919044495),
 ('Russlands_Staatschef', 0.6975833177566528),
 ('Wladimir_Putin', 0.6830098628997803),
 ('russischen_Praesidenten', 0.6809293031692505),
 ('US-Praesident_Barack', 0.6808925271034241),
 ('US-Aussenminister_Kerry', 0.6703359484672546),
 ('Kremlchef_Wladimir', 0.6681345105171204),
 ('Russlands_Praesident', 0.6653517484664917)]

In [6]:
wv.most_similar(positive=["Merkel", "USA"], negative=["Deutschland"])

[('Kanzlerin_Merkel', 0.6972562074661255),
 ('Merkel_CDU', 0.689968466758728),
 ('Kanzlerin', 0.6769890189170837),
 ('US-Praesident_Obama', 0.6683844327926636),
 ('Obama', 0.650946319103241),
 ('Russlands_Staatschef', 0.6507245898246765),
 ('Bundeskanzlerin', 0.6503757834434509),
 ('Ueberwachung_Handys', 0.6279087662696838),
 ('Angela_Merkel', 0.6278500556945801),
 ('Rande_G20-Gipfels', 0.6272603273391724)]

In [9]:
wv.most_similar(positive=["Mutter"])

[('Grossmutter', 0.8721050024032593),
 ('Schwester', 0.8409271240234375),
 ('Tochter', 0.8314520716667175),
 ('Vater', 0.825255274772644),
 ('Tante', 0.8161165714263916),
 ('Oma', 0.8082747459411621),
 ('Frau', 0.7920665740966797),
 ('vierjaehrige_Tochter', 0.7915295362472534),
 ('Stiefmutter', 0.7876833081245422),
 ('Pflegemutter', 0.78443843126297)]

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
import numpy as np

# Prepare embedding matrix
embedding_dim = wv.vector_size
vocab_size = len(wv.key_to_index)

embedding_matrix = np.zeros((vocab_size, embedding_dim))
for i, word in enumerate(wv.key_to_index):
	embedding_matrix[i] = wv[word]

# Create Keras Embedding layer with pretrained weights
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, weights=[embedding_matrix], trainable=False))